In [239]:
#import required packages
import requests
import json
import os
import sqlite3 
import pandas as pd
from decouple import config
from geopy.geocoders import Nominatim
import pprint

### Feature Engineering - Part 2

In [225]:
# df = pd.read_csv("houses_cleaned_baseline_crimedata.csv", index_col = 0)
# df = pd.read_csv("houses_cleaned_baselineNanDropped_crimedata.csv", index_col = 0)
df = pd.read_csv("houses_cleaned_baselineKNN_crimedata.csv", index_col = 0)

df.head()

,Unnamed: 0.1,final_price,bathrooms,parking,description,type,full_link,full_address,lat,long,...,OBJECTID,Neighbourhood,Hood_ID,Shape__Area,Shape__Length,Assault_Rate,Auto_Theft_Rate,Break_andEnter_Rate,Robbery_Rate,Shootings_Rate
0,0,855000,2,1,"Luxurious And Spacious Murano Tower. 2+1, 2 Ba...",Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"38 Grenville St, Toronto , Ontario, Canada",43.661896,-79.385748,...,94,Bay Street Corridor,76,1.809355e+06,6933.183322,2511.6408,100.95383,532.97306,371.57816,12.331211
1,85,785000,1,1,Five Condos - Stunning Bright Corner Unit In A...,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"5 St Joseph St, Toronto , Ontario, Canada",43.665753,-79.385579,...,94,Bay Street Corridor,76,1.809355e+06,6933.183322,2511.6408,100.95383,532.97306,371.57816,12.331211
2,154,718000,2,0,"Spacious 2 Bedrooms, 2 Full Baths, Over 1000 S...",Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"633 Bay St, Toronto , Ontario, Canada",43.656674,-79.383443,...,94,Bay Street Corridor,76,1.809355e+06,6933.183322,2511.6408,100.95383,532.97306,371.57816,12.331211
3,161,968000,2,1,Beautiful Burano On Bay Is A Diamond In The Sk...,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"832 Bay St, Toronto , Ontario, Canada",43.661927,-79.386511,...,94,Bay Street Corridor,76,1.809355e+06,6933.183322,2511.6408,100.95383,532.97306,371.57816,12.331211
4,216,650000,2,1,*Smart & Functional One Bdrm + Den + Extra Bat...,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"111 Elizabeth St, Toronto , Ontario, Canada",43.655055,-79.384721,...,94,Bay Street Corridor,76,1.809355e+06,6933.183322,2511.6408,100.95383,532.97306,371.57816,12.331211


In [212]:
neighbourhoods = df.Neighbourhood.unique()
neighbourhoods

array(['Bay Street Corridor', 'Islington-City Centre West',
       'Mount Pleasant West', 'Niagara', 'New Toronto',
       'Lawrence Park South', 'Glenfield-Jane Heights',
       'Mount Pleasant East', 'Malvern', 'Junction Area',
       'Wexford/Maryvale', 'Cabbagetown-South St.James Town',
       'St.Andrew-Windfields', 'Banbury-Don Mills', 'Milliken',
       'Mimico (includes Humber Bay Shores)', 'West Hill',
       'Birchcliffe-Cliffside', 'Dorset Park', 'Yorkdale-Glen Park',
       'Church-Yonge Corridor', 'Rockcliffe-Smythe', "O'Connor-Parkview",
       'York University Heights', 'South Parkdale',
       'Downsview-Roding-CFB', 'Trinity-Bellwoods',
       'Dovercourt-Wallace Emerson-Junction', 'Willowdale West',
       'Greenwood-Coxwell', 'Markland Wood', 'Scarborough Village',
       'Victoria Village', 'East End-Danforth', 'North St.James Town',
       'Ionview', 'Rustic', 'Waterfront Communities-The Island',
       'Mount Dennis', 'Oakwood Village', 'Annex', 'Flemingdon Park',

In [213]:
#get coordinates of every neighbourhood

hoods = {}
hood_nan = []
for neighbourhood in neighbourhoods:
    try:
        address = neighbourhood + ', ' + 'Toronto, Canada'
        geolocator = Nominatim(user_agent="toronto_explorer")
        location = geolocator.geocode(address)
        lat = location.latitude
        lng = location.longitude
        hoods[neighbourhood] = str(lat)+","+ str(lng)
        
    except:
        print (neighbourhood)
        hood_nan.append(neighbourhood)

Lawrence Park South
Glenfield-Jane Heights
Junction Area
Wexford/Maryvale
Cabbagetown-South St.James Town
St.Andrew-Windfields
Mimico (includes Humber Bay Shores)
Birchcliffe-Cliffside
O'Connor-Parkview
Downsview-Roding-CFB
Greenwood-Coxwell
Waterfront Communities-The Island
Agincourt South-Malvern West
Bedford Park-Nortown
Newtonbrook East
Humbermede
Mount Olive-Silverstone-Jamestown
Princess-Rosethorn
Briar Hill-Belgravia
Kingsview Village-The Westway
Elms-Old Rexdale
Bridle Path-Sunnybrook-York Mills
Clairlea-Birchmount
Woodbine Corridor
Newtonbrook West
Woodbine-Lumsden
Pelmo Park-Humberlea
Parkwoods-Donalda
Lansing-Westgate
Forest Hill South
Westminster-Branson
Brookhaven-Amesbury
Eringate-Centennial-West Deane
Blake-Jones
Thistletown-Beaumond Heights
Weston-Pellam Park
Willowridge-Martingrove-Richview
Beechborough-Greenbrook
Caledonia-Fairbank


In [214]:
#Manually adding coordinates for each missing neighbourhood

hoods['Lawrence Park South'] = str(43.7121)+","+ str(-79.4117)
hoods['Glenfield-Jane Heights'] = str(43.74675)+","+ str(-79.51504)
hoods['Junction Area'] = str(43.6655)+","+ str(-79.4721)
hoods['Wexford/Maryvale'] = str(43.7613)+","+ str(-79.3008)
hoods['Cabbagetown-South St.James Town'] = str(43.6664)+","+ str(-79.3629)
hoods['St.Andrew-Windfields'] = str(43.7548)+","+ str(-79.3855)
hoods['Mimico (includes Humber Bay Shores)'] = str(43.6146)+","+ str(-79.4957)
hoods['Birchcliffe-Cliffside'] = str(43.6875)+","+ str(-79.2694)
hoods["O'Connor-Parkview"] = str(43.7061)+","+ str(-79.3102)
hoods['Downsview-Roding-CFB'] = str(43.73329)+","+ str( -79.49049)
hoods['Greenwood-Coxwell'] = str(43.6721)+","+ str(-79.3228)
hoods['Waterfront Communities-The Island'] = str(43.644)+","+ str(-79.380218)
hoods['Agincourt South-Malvern West'] = str(43.789)+","+ str(-79.26941)
hoods['Bedford Park-Nortown'] = str(43.7303)+","+ str(-79.4114)
hoods['Newtonbrook East'] = str(43.78333)+","+ str(-79.41666)
hoods['Humbermede'] = str(43.7390)+","+ str(-79.5394)
hoods['Mount Olive-Silverstone-Jamestown'] = str(43.74687)+","+ str(-79.58726)
hoods['Princess-Rosethorn'] = str(43.6700)+","+ str(-79.5477)
hoods['Briar Hill-Belgravia'] = str(43.7037)+","+ str(-79.4524)
hoods['Kingsview Village-The Westway'] = str(43.69944)+","+ str(-79.54924)
hoods['Elms-Old Rexdale'] = str(43.72429)+","+ str(-79.54956)
hoods['Bridle Path-Sunnybrook-York Mills'] = str(43.7359)+","+ str(-79.3719)
hoods['Clairlea-Birchmount'] = str(43.7185)+","+ str(-79.2914)
hoods['Woodbine Corridor'] = str(43.682088)+","+ str(-79.3032772)
hoods['Newtonbrook West'] = str(43.7871)+","+ str(-79.4173)
hoods['Woodbine-Lumsden'] = str(43.6922)+","+ str(-79.3099)
hoods['Pelmo Park-Humberlea'] = str(43.7104)+","+ str(-79.5219)
hoods['Parkwoods-Donalda'] = str(43.7607)+","+ str(-79.3277)
hoods['Lansing-Westgate'] = str(43.7590)+","+ str(-79.4226)
hoods['Forest Hill South'] = str(43.6932)+","+ str(-79.4126)
hoods['Westminster-Branson'] = str(43.7856)+","+ str(-79.4511)
hoods['Brookhaven-Amesbury'] = str(43.6969)+","+ str(-79.4938)
hoods['Eringate-Centennial-West Deane'] = str(43.6599)+","+ str(-79.5833)
hoods['Blake-Jones'] = str(43.6759)+","+ str(-79.3381)
hoods['Thistletown-Beaumond Heights'] = str(43.7372)+","+ str(-79.5653)
hoods['Weston-Pellam Park'] = str(43.6716)+","+ str(-79.4577)
hoods['Willowridge-Martingrove-Richview'] = str(43.6789276752)+","+ str(-79.5715452921)
hoods['Beechborough-Greenbrook'] = str(43.6943)+","+ str(-79.4748)
hoods['Caledonia-Fairbank'] = str(43.6899)+","+ str(-79.4552)

In [227]:
#adding neighbourhood coordinates to dataframe

df_hoods = pd.DataFrame({'Neighbourhood':hoods.keys(),'Neighbourhood_Coordin':hoods.values()})

In [228]:
df_hoods

,Neighbourhood,Neighbourhood_Coordin
0,Bay Street Corridor,"43.6727982,-79.3907344"
1,Islington-City Centre West,"43.6412369,-79.5390644"
2,Mount Pleasant West,"43.7084166,-79.3901348"
3,Niagara,"43.6440753,-79.4086982"
4,New Toronto,"43.6007625,-79.505264"
...,...,...
135,Thistletown-Beaumond Heights,"43.7372,-79.5653"
136,Weston-Pellam Park,"43.6716,-79.4577"
137,Willowridge-Martingrove-Richview,"43.6789276752,-79.5715452921"
138,Beechborough-Greenbrook,"43.6943,-79.4748"


### Foursquare API

In [229]:
fsquare_clientid = str(config('fsquare_clientid'))
fsquare_key = str(config('fsquare_key'))
fsquare_clientsecret = str(config('fsquare_clientsecret'))
version = '20220810'
limit = 100000

In [230]:
url = "https://api.foursquare.com/v3/places/search"

headers = {"Accept": "application/json"}
headers['Authorization'] = fsquare_key
categories = ['13065','19043','16032','12057','10000','17115','15014']

for cat in categories:
    
    df_hoods[cat] = 0*len(df_hoods)

for row in range(len(df_hoods)):
    
    coordin = df_hoods['Neighbourhood_Coordin'].iloc[row]
    print (row)
    
    for cat in categories:
        
        params = {'ll':coordin,'radius':'1000','categories':cat, 'limit':50}
        response = requests.get(url, headers=headers, params = params)
        total = len(response.json()['results'])
        df_hoods[cat].iloc[row] = total

0


C:\Users\bevli\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139


In [231]:
df_hoods

,Neighbourhood,Neighbourhood_Coordin,13065,19043,16032,12057,10000,17115,15014
0,Bay Street Corridor,"43.6727982,-79.3907344",50,0,25,25,50,1,10
1,Islington-City Centre West,"43.6412369,-79.5390644",37,1,2,5,6,0,1
2,Mount Pleasant West,"43.7084166,-79.3901348",50,0,10,23,28,1,1
3,Niagara,"43.6440753,-79.4086982",50,0,14,13,33,2,2
4,New Toronto,"43.6007625,-79.505264",31,0,6,4,10,0,0
...,...,...,...,...,...,...,...,...,...
135,Thistletown-Beaumond Heights,"43.7372,-79.5653",29,0,0,3,1,0,0
136,Weston-Pellam Park,"43.6716,-79.4577",39,0,10,6,19,0,0
137,Willowridge-Martingrove-Richview,"43.6789276752,-79.5715452921",4,0,3,4,2,0,0
138,Beechborough-Greenbrook,"43.6943,-79.4748",23,0,2,6,4,2,2


In [232]:
#renaming categories columns
categories = ['13065','19043','16032','12057','10000','17115','15014']
categories_names = ['Restaurant','Bus_Stops','Parks','Schools','Entertainment_Services','Shopping_Plazas','Hospitals']
categories_dict = {}

for i in range(len(categories)):
    categories_dict[categories[i]] = categories_names[i]

df_hoods = df_hoods.rename(columns = categories_dict)


In [233]:
df_hoods

,Neighbourhood,Neighbourhood_Coordin,Restaurant,Bus_Stops,Parks,Schools,Entertainment_Services,Shopping_Plazas,Hospitals
0,Bay Street Corridor,"43.6727982,-79.3907344",50,0,25,25,50,1,10
1,Islington-City Centre West,"43.6412369,-79.5390644",37,1,2,5,6,0,1
2,Mount Pleasant West,"43.7084166,-79.3901348",50,0,10,23,28,1,1
3,Niagara,"43.6440753,-79.4086982",50,0,14,13,33,2,2
4,New Toronto,"43.6007625,-79.505264",31,0,6,4,10,0,0
...,...,...,...,...,...,...,...,...,...
135,Thistletown-Beaumond Heights,"43.7372,-79.5653",29,0,0,3,1,0,0
136,Weston-Pellam Park,"43.6716,-79.4577",39,0,10,6,19,0,0
137,Willowridge-Martingrove-Richview,"43.6789276752,-79.5715452921",4,0,3,4,2,0,0
138,Beechborough-Greenbrook,"43.6943,-79.4748",23,0,2,6,4,2,2


In [234]:
#Adding individual neighbourhood coordinates to df

df = pd.merge(df, df_hoods, left_on=['Neighbourhood'],right_on=['Neighbourhood'], how = 'inner')

In [235]:
df

,Unnamed: 0.1,final_price,bathrooms,parking,description,type,full_link,full_address,lat,long,...,Robbery_Rate,Shootings_Rate,Neighbourhood_Coordin,Restaurant,Bus_Stops,Parks,Schools,Entertainment_Services,Shopping_Plazas,Hospitals
0,0,855000,2,1,"Luxurious And Spacious Murano Tower. 2+1, 2 Ba...",Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"38 Grenville St, Toronto , Ontario, Canada",43.661896,-79.385748,...,371.578160,12.331211,"43.6727982,-79.3907344",50,0,25,25,50,1,10
1,85,785000,1,1,Five Condos - Stunning Bright Corner Unit In A...,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"5 St Joseph St, Toronto , Ontario, Canada",43.665753,-79.385579,...,371.578160,12.331211,"43.6727982,-79.3907344",50,0,25,25,50,1,10
2,154,718000,2,0,"Spacious 2 Bedrooms, 2 Full Baths, Over 1000 S...",Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"633 Bay St, Toronto , Ontario, Canada",43.656674,-79.383443,...,371.578160,12.331211,"43.6727982,-79.3907344",50,0,25,25,50,1,10
3,161,968000,2,1,Beautiful Burano On Bay Is A Diamond In The Sk...,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"832 Bay St, Toronto , Ontario, Canada",43.661927,-79.386511,...,371.578160,12.331211,"43.6727982,-79.3907344",50,0,25,25,50,1,10
4,216,650000,2,1,*Smart & Functional One Bdrm + Den + Extra Bat...,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"111 Elizabeth St, Toronto , Ontario, Canada",43.655055,-79.384721,...,371.578160,12.331211,"43.6727982,-79.3907344",50,0,25,25,50,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15229,11499,1430000,5,2,Custom Built! Brand New Never Been Lived!Cover...,Detached,https://www.zoocasa.com/toronto-on-real-estate...,"33 Cosens St, Toronto , Ontario, Canada",43.796526,-79.173685,...,127.159984,9.079792,"43.7901172,-79.1733344",9,0,1,6,0,1,0
15230,12791,900000,3,4,"Immaculate 4 Bed, 3 Bath Raised Bungalow With ...",Detached,https://www.zoocasa.com/toronto-on-real-estate...,"57 Watson St, Toronto , Ontario, Canada",43.786423,-79.173985,...,127.159984,9.079792,"43.7901172,-79.1733344",9,0,1,6,0,1,0
15231,14732,805000,4,2,"4 Bedroom, 4 Washroom Home In Prestigious High...",Detached,https://www.zoocasa.com/toronto-on-real-estate...,"44 Fishery Rd, Toronto , Ontario, Canada",43.791859,-79.175068,...,127.159984,9.079792,"43.7901172,-79.1733344",9,0,1,6,0,1,0
15232,14950,740000,4,4,You Will Love The Location Of This All Brick F...,Detached,https://www.zoocasa.com/toronto-on-real-estate...,"76 Muirbank Blvd, Toronto , Ontario, Canada",43.795167,-79.158194,...,127.159984,9.079792,"43.7901172,-79.1733344",9,0,1,6,0,1,0


In [236]:
# df.to_csv('houses_cleaned_baseline_crimedata_foursquare.csv')
# df.to_csv('houses_cleaned_baselineNanDropped_crimedata_foursquare.csv')
df.to_csv('houses_cleaned_baselineKNN_crimedata_foursquare.csv')